[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rupaltotale/Movie-Genre-Predictor/blob/master/Cleaning%20Data.ipynb)
# Predicting Movie Genres: Data Cleaning

In this notebook, we clean the data by converting the datasets we fetched in Notebook 1 in CSV format to a DataFrame, clean the data by transforming columns to be useful for our purposes, and finally merge the DataFrames to produce one with all the features that define a movie (inclusing genres). 

## Make all neccessary imports and setup the environment

In [0]:
import pandas as pd
from google.colab import drive

In [2]:
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
data_dir = "drive/My Drive/301 Project/Data/"

## Clean and convert Kaggle data set to DataFrame

In [0]:
print("Files for Kaggle: \n")
!ls "drive/My Drive/301 Project/Data/" | grep .csv

Files for Kaggle: 

cleaned_dataframe.csv
credits.csv
keywords.csv
links.csv
links_small.csv
movies_metadata.csv
ratings.csv
ratings_small.csv


### Convert user_ratings.csv to a DataFrame, clean it, and extract movie ids from it

In [0]:
df_ratings = pd.read_csv(data_dir + "ratings.csv")
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [0]:
# Add average user rating as a column
df_ratings_clean = df_ratings.join(
    df_ratings.groupby("movieId")["rating"].mean(), 
    on='movieId', 
    rsuffix='_average'
)

# Setup: Add users who like a given movie as a column
def convert_to_string(lst):
  ids = ""
  for i in lst:
    ids += str(i) + " "
  return ids

good_user_ids = (df_ratings_clean[df_ratings_clean["rating"] > 3]
                 .groupby("movieId")["userId"]
                 .apply(list))

df_ratings_clean = (
    # Add users who like a given movie as a column
    df_ratings_clean.join(
      good_user_ids.apply(convert_to_string), 
      on='movieId', 
      rsuffix='_who_like'
    )
    # Drop userId column and rating column as they are no longer needed
    .drop(columns=["userId", "rating", "timestamp"], axis = 1)
    # Movie ids should be unique now as each row is a distinct movie
    .drop_duplicates(subset='movieId', keep='first')
    # To be consistent with other datasets, rename movieId
    .rename(columns={'movieId': 'id'})
)
df_ratings_clean

,id,rating_average,userId_who_like
0,110,4.016057,11 22 24 30 34 49 56 58 63 64 65 68 80 82 88 8...
1,147,3.595933,1 24 70 142 463 638 690 726 778 831 882 884 88...
2,858,4.339811,1 3 5 12 20 24 27 28 30 37 47 49 50 56 62 63 6...
3,1221,4.263475,1 4 12 20 24 27 28 34 47 49 50 56 63 65 89 120...
4,1246,3.912803,1 9 12 24 49 62 64 68 82 97 98 106 115 120 132...
...,...,...,...
26023429,159050,5.000000,270887
26023430,159053,5.000000,270887
26023482,165649,4.000000,270887
26023515,171051,4.000000,270887


In [0]:
movie_ids_dict = set(df_ratings_clean["id"].value_counts().index)
print("Number of movies being considered so far:", len(movie_ids_dict))

Number of movies being considered so far: 45115


### Convert credits.csv to a data frame and clean it

In [0]:
df_credits = pd.read_csv(data_dir + "credits.csv")
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [0]:
"""
Crew ==> Get names of Screenplay, Director, Composer
Cast ==> Get all names, underscore instead of space for each member's full name,
         space seperated
"""
def clean_credits(obs):
  # Crew
  crew_members = eval(obs["crew"])
  for person in crew_members:
    if person["job"] == "Screenplay":
      obs["Screenplay"] = person["name"].replace(" ", "_")
    elif person["job"] == "Director":
      obs["Director"] =  person["name"].replace(" ", "_")
    elif person["job"] == "Original Music Composer":
      obs["Composer"] =  person["name"].replace(" ", "_")
  # Cast
  cast = eval(obs["cast"])
  names = ""
  for person in cast:
    names += person["name"].replace(" ", "_") + " "
  obs["cast"] = names
  return obs

df_credits_clean = (
    df_credits[df_credits["id"].isin(movie_ids_dict)]
               .apply(clean_credits, axis=1)
               # Crew is no longer needed
               .drop(
                  columns=["crew"], 
                  axis=1)
               )
df_credits_clean

,Composer,Director,Screenplay,cast,id
0,NaN,John_Lasseter,Alec_Sokolow,Tom_Hanks Tim_Allen Don_Rickles Jim_Varney Wal...,862
1,James_Horner,Joe_Johnston,Jim_Strain,Robin_Williams Jonathan_Hyde Kirsten_Dunst Bra...,8844
5,Elliot_Goldenthal,Michael_Mann,Michael_Mann,Al_Pacino Robert_De_Niro Val_Kilmer Jon_Voight...,949
9,Eric_Serra,Martin_Campbell,Bruce_Feirstein,Pierce_Brosnan Sean_Bean Izabella_Scorupco Fam...,710
14,NaN,Renny_Harlin,Robert_King,Geena_Davis Matthew_Modine Frank_Langella Maur...,1408
...,...,...,...,...,...
45416,NaN,Sergey_Ivanov,NaN,Kristina_Asmus Nikita_Efremov Artyom_Tkachenko...,98604
45443,NaN,Bořivoj_Zeman,František_Vlček,Helena_Vondráčková Václav_Neckář Jaroslav_Marv...,5589
45446,NaN,Uwe_Boll,NaN,Lauren_Holly Luke_Perry Steve_Bacic Cole_Heppe...,45527
45460,NaN,Georges_Méliès,NaN,Georges_Méliès,49280


**Note**: The reason this DataFrame only consists of ~7571 observations even though we were considering around 45,000 movies in the previous section is because not all movie ids in user_ratings appear (or are defined) in other data sets. Same logic applies for the next two DataSets.

### Convert keywords.csv to a data frame and clean it

In [0]:
df_keywords = pd.read_csv(data_dir + "keywords.csv")
df_keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [0]:
"""
Clean columns:
  keywords ==> keywords.name with underscore replacing space in each keyword,
               space seperated
"""
def clean_keywords(obs):
  keywords_as_string = ""
  for keyword in eval(obs["keywords"]):
    keywords_as_string += keyword["name"].replace(" ", "_") + " "
  obs["keywords"] = keywords_as_string
  return obs
  
df_keywords_clean = (
    df_keywords[df_keywords["id"].isin(movie_ids_dict)]
               .apply(clean_keywords, axis=1)
)
df_keywords_clean.head()

,id,keywords
0,862,jealousy toy boy friendship friends rivalry bo...
1,8844,board_game disappearance based_on_children's_b...
5,949,robbery detective bank obsession chase shootin...
9,710,cuba falsely_accused secret_identity computer_...
14,1408,exotic_island treasure map ship scalp pirate


### Convert movies_metadata.csv to a DataFrame and clean it

In [0]:
df_meta = pd.read_csv(data_dir + "movies_metadata.csv", low_memory=False)
# This are corrupt observation (where movie id is not an integer), 
# so they need to be weeded out
df_meta = df_meta[pd.to_numeric(df_meta['id'], errors='coerce').notnull()]
df_meta["id"] = df_meta["id"].astype(int)
df_meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


#### Get all genres from genres.csv

In [9]:
df_genres = pd.read_csv(data_dir + "genres.csv").drop(columns=["Unnamed: 0"])
all_genres = list(df_genres["name"])
all_genres

['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

#### Clean df_meta
The code cell below creates new columns in `df_meta` for each genre. 

In [0]:
for genre in all_genres:
  df_meta[genre] = 0
print("All columns of df_meta_clean", df_meta.columns)

All columns of df_meta_clean Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'Animation', 'Horror', 'Mystery',
       'Fantasy', 'Romance', 'TV Movie', 'Family', 'Documentary', 'Western',
       'Science Fiction', 'Foreign', 'Crime', 'Adventure', 'Thriller',
       'Comedy', 'War', 'Drama', 'History', 'Action', 'Music'],
      dtype='object')


The code cell below cleans each column of df_meta_clean as described in the comment block. 

In [0]:
"""
Clean columns:
  belongs_to_collection ==> collection.name | NaN
  production_companies ==> space seperated, underscored instead of space for 
    distinct companies. 
  Genres ==> Seperate columns
    If movie is of the genre type, column value is 1, else 0.

Keep columns: budget, id, original_title,	overview,	popularity, 
    production_companies, revenue, runtime, tagline, vote_average, vote_count,
    [genres]
"""
def clean_meta(obs):
  # Genre columns
  genres = eval(obs["genres"])
  for genre in genres:
    obs[genre["name"]] = 1
  # Production companies
  production_companies = ""
  for company in eval(obs["production_companies"]):
    production_companies += company["name"].replace(" ", "_") + " "
  obs["production_companies"] = production_companies
  # Collections
  if not pd.isna(obs["belongs_to_collection"]):
    obs["collection"] = eval(
          obs["belongs_to_collection"]
        )["name"].replace(" ", "_")
  return obs

# Define what columns to keep in df_meta_clean
keep_columns = all_genres + ["id", "original_title", "budget", 
                             "overview",	"popularity", "production_companies", 
                             "revenue", "runtime", "tagline", "vote_average", 
                             "vote_count", "collection"]

df_meta_clean = (
    df_meta[df_meta["id"].isin(movie_ids_dict)]
      .apply(clean_meta, axis=1)
      # The dataset seems to have duplicate observations that need to be dropped
      .drop_duplicates(subset='id', keep='first')
      [keep_columns]
)
df_meta_clean.head()

,Animation,Horror,Mystery,Fantasy,Romance,TV Movie,Family,Documentary,Western,Science Fiction,Foreign,Crime,Adventure,Thriller,Comedy,War,Drama,History,Action,Music,id,original_title,budget,overview,popularity,production_companies,revenue,runtime,tagline,vote_average,vote_count,collection
0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,862,Toy Story,30000000,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar_Animation_Studios,373554033.0,81.0,NaN,7.7,5415.0,Toy_Story_Collection
1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,8844,Jumanji,65000000,When siblings Judy and Peter discover an encha...,17.015539,TriStar_Pictures Teitler_Film Interscope_Commu...,262797249.0,104.0,Roll the dice and unleash the excitement!,6.9,2413.0,NaN
5,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,949,Heat,60000000,"Obsessive master thief, Neil McCauley leads a ...",17.924927,Regency_Enterprises Forward_Pass Warner_Bros.,187436818.0,170.0,A Los Angeles Crime Saga,7.7,1886.0,NaN
9,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,710,GoldenEye,58000000,James Bond must unmask the mysterious head of ...,14.686036,United_Artists Eon_Productions,352194034.0,130.0,No limits. No fears. No substitutes.,6.6,1194.0,James_Bond_Collection
14,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1408,Cutthroat Island,98000000,"Morgan Adams and her slave, William Shaw, are ...",7.284477,Le_Studio_Canal+ Laurence_Mark_Productions Met...,10017322.0,119.0,The Course Has Been Set. There Is No Turning B...,5.7,137.0,NaN


### Merge all the DataFrames into a single DataFrame, and save (as csv) to drive

In [0]:
from functools import reduce
data_frames = [df_meta_clean, 
               df_credits_clean, 
               df_keywords_clean, 
               df_ratings_clean]
df_merged = reduce(lambda left, right: pd.merge(left, right, on=['id'], 
                                                how='inner'), 
                   data_frames)

print("Number of movies:", len(df_merged))
print("Columns of merged DataFrame:", df_merged.columns)
df_merged.head()

Number of movies: 7679
Columns of merged DataFrame: Index(['Animation', 'Horror', 'Mystery', 'Fantasy', 'Romance', 'TV Movie',
       'Family', 'Documentary', 'Western', 'Science Fiction', 'Foreign',
       'Crime', 'Adventure', 'Thriller', 'Comedy', 'War', 'Drama', 'History',
       'Action', 'Music', 'id', 'original_title', 'budget', 'overview',
       'popularity', 'production_companies', 'revenue', 'runtime', 'tagline',
       'vote_average', 'vote_count', 'collection', 'Composer', 'Director',
       'Screenplay', 'cast', 'keywords', 'rating_average', 'userId_who_like'],
      dtype='object')


,Animation,Horror,Mystery,Fantasy,Romance,TV Movie,Family,Documentary,Western,Science Fiction,Foreign,Crime,Adventure,Thriller,Comedy,War,Drama,History,Action,Music,id,original_title,budget,overview,popularity,production_companies,revenue,runtime,tagline,vote_average,vote_count,collection,Composer,Director,Screenplay,cast,keywords,rating_average,userId_who_like
0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,862,Toy Story,30000000,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar_Animation_Studios,373554033.0,81.0,NaN,7.7,5415.0,Toy_Story_Collection,NaN,John_Lasseter,Alec_Sokolow,Tom_Hanks Tim_Allen Don_Rickles Jim_Varney Wal...,jealousy toy boy friendship friends rivalry bo...,3.598930,2103 6177 6525 8659 9328 9682 11214 13839 1523...
1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,8844,Jumanji,65000000,When siblings Judy and Peter discover an encha...,17.015539,TriStar_Pictures Teitler_Film Interscope_Commu...,262797249.0,104.0,Roll the dice and unleash the excitement!,6.9,2413.0,NaN,James_Horner,Joe_Johnston,Jim_Strain,Robin_Williams Jonathan_Hyde Kirsten_Dunst Bra...,board_game disappearance based_on_children's_b...,3.760163,7016 7144 8659 9547 13399 17808 24266 25209 26...
2,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,949,Heat,60000000,"Obsessive master thief, Neil McCauley leads a ...",17.924927,Regency_Enterprises Forward_Pass Warner_Bros.,187436818.0,170.0,A Los Angeles Crime Saga,7.7,1886.0,NaN,Elliot_Goldenthal,Michael_Mann,Michael_Mann,Al_Pacino Robert_De_Niro Val_Kilmer Jon_Voight...,robbery detective bank obsession chase shootin...,3.905544,174 346 362 384 523 557 609 624 773 825 959 11...
3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,710,GoldenEye,58000000,James Bond must unmask the mysterious head of ...,14.686036,United_Artists Eon_Productions,352194034.0,130.0,No limits. No fears. No substitutes.,6.6,1194.0,James_Bond_Collection,Eric_Serra,Martin_Campbell,Bruce_Feirstein,Pierce_Brosnan Sean_Bean Izabella_Scorupco Fam...,cuba falsely_accused secret_identity computer_...,2.740334,2073 2709 2967 4241 4389 4916 5463 5811 5980 8...
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1408,Cutthroat Island,98000000,"Morgan Adams and her slave, William Shaw, are ...",7.284477,Le_Studio_Canal+ Laurence_Mark_Productions Met...,10017322.0,119.0,The Course Has Been Set. There Is No Turning B...,5.7,137.0,NaN,NaN,Renny_Harlin,Robert_King,Geena_Davis Matthew_Modine Frank_Langella Maur...,exotic_island treasure map ship scalp pirate,3.710181,15 65 68 98 115 151 153 193 231 288 326 340 34...


#### Convert merged data to CSV and save to google drive

In [0]:
df_merged.to_csv("cleaned_dataframe.csv")
!ls

cleaned_dataframe.csv  drive  sample_data


In [0]:
!cp -r cleaned_dataframe.csv "drive/My Drive/301 Project/Data"